## 1. Install and import the libraries

In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm
%pip install gensim
%pip install pyLDAvis

In [3]:
# libraries for text cleaning and manipulation
import ast
from bs4 import BeautifulSoup
import unicodedata
import re
import boto3
import pandas as pd

In [4]:
# to show progress bar
from tqdm import tqdm
# for pretty printing
import pprint

In [5]:
# nlp library
import spacy

In [6]:
# for topic modelling
from gensim.models.phrases import Phrases, Phraser, ENGLISH_CONNECTOR_WORDS
from gensim import corpora
from gensim.models import ldamodel, TfidfModel

## 2. Read and clean the data

In [7]:
# Getting data from AWS S3 bucket
s3 = boto3.client('s3')
obj = s3.get_object(Bucket = 'process-news',Key = 'techcrunch_data.csv')

In [8]:
techcrunch_data = pd.read_csv(obj['Body'])

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35,47,48,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
techcrunch_data.shape

(32634, 54)

In [10]:
techcrunch_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'date', 'date_gmt', 'guid',
       'modified', 'modified_gmt', 'slug', 'status', 'type', 'link', 'title',
       'content', 'excerpt', 'author', 'featured_media', 'comment_status',
       'ping_status', 'sticky', 'template', 'format', 'meta', 'categories',
       'tags', 'crunchbase_tag', 'tc_stories_tax', 'tc_ec_category',
       'tc_event', 'jetpack_featured_media_url', 'shortlink', 'rapidData',
       'premiumContent', 'premiumCutoffPercent', 'featured', 'subtitle',
       'seoTitle', 'editorialContentProvider', 'seoDescription',
       'tc_cb_mapping', 'associatedEvent', 'event', 'authors',
       'hide_featured_image', 'canonical_url', '_links', '_embedded',
       'tc_show', 'vidible_hash', 'thumbnail_url', 'videoDuration',
       'parentLink', 'accessLevel', 'slides'],
      dtype='object')

In [11]:
# Cleaning the data
techcrunch_data["clean_title"] =  [ast.literal_eval(x)['rendered'] for x in techcrunch_data["title"]]
techcrunch_data["clean_content"] =  [ast.literal_eval(x)['rendered'] for x in techcrunch_data["content"]]

In [12]:
# Concatenate title and content in one string
techcrunch_data['clean_text'] = techcrunch_data["clean_title"] + ' ' + techcrunch_data["clean_content"]

In [13]:
# getting a text from the markup
techcrunch_data["clean_text"] = [ BeautifulSoup(text, 'lxml').text for text in techcrunch_data["clean_text"]]

In [14]:
# remove unicode characters
techcrunch_data["clean_text"] = [unicodedata.normalize('NFKD', x) for x in techcrunch_data["clean_text"]]

In [ ]:
# replace /n, /t and /r
techcrunch_data["clean_text"] = [x.replace('\n',' ').replace('\t', ' ').replace('\r', ' ') for x in techcrunch_data["clean_text"]]